In [1]:
from importlib import reload
# import VAE as VAE
import VAE.DataLoader as DataLoader
import VAE.Encoder as Encoder
import VAE.Decoder as Decoder
import VAE.ConvSeq2Seq as ConvSeq2Seq
import VAE.Discriminator as Discriminator
import torch
from torch import nn
import torch.optim as optim
import math

In [12]:
reload(Encoder)
reload(Decoder)
reload(ConvSeq2Seq)
reload(Discriminator)
reload(DataLoader)

<module 'VAE.DataLoader' from '/home/wichen/repos/CS7643_DL/Project/VAE/DataLoader.py'>

In [9]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [6]:
device='cpu'

In [3]:
dloader = DataLoader.DataLoader(50, 25, './data/h3.6m/dataset')

Reading subject 1, action walking, subaction 1
Reading subject 1, action walking, subaction 2
Reading subject 1, action eating, subaction 1
Reading subject 1, action eating, subaction 2
Reading subject 1, action smoking, subaction 1
Reading subject 1, action smoking, subaction 2
Reading subject 1, action discussion, subaction 1
Reading subject 1, action discussion, subaction 2
Reading subject 1, action directions, subaction 1
Reading subject 1, action directions, subaction 2
Reading subject 1, action greeting, subaction 1
Reading subject 1, action greeting, subaction 2
Reading subject 1, action phoning, subaction 1
Reading subject 1, action phoning, subaction 2
Reading subject 1, action posing, subaction 1
Reading subject 1, action posing, subaction 2
Reading subject 1, action purchases, subaction 1
Reading subject 1, action purchases, subaction 2
Reading subject 1, action sitting, subaction 1
Reading subject 1, action sitting, subaction 2
Reading subject 1, action sittingdown, subacti

In [10]:
print('Memory usage before construct network: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

Memory usage before construct network: 0 bytes => 0.0 GB


In [13]:
lt_encoder = Encoder.Encoder(16,enc_shape=[None, 49, 54, 1], enc_dim_desc={ 'hidden_num': 512,'class_num': 15 })

st_encoder = Encoder.Encoder(16,enc_shape=[None, 20, 54, 1], enc_dim_desc={ 'hidden_num': 512})
decoder = Decoder.Decoder(st_encoder)

generator = ConvSeq2Seq.ConvSeq2Seq(lt_encoder, decoder, window_length=20, device=device)

d_encoder = Encoder.Encoder(32, enc_shape=[None, 75,54,1], enc_dim_desc={ 'hidden_num': 512})
discriminator = Discriminator.Discriminator(32, d_encoder).to(device)

print('Memory usage after construct network: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

Memory usage after construct network: 1566820352 bytes => 1.566820352 GB


In [14]:
G_criterion = nn.MSELoss()
D_criterion = nn.BCELoss()

In [15]:
# Setup Adam optimizers for both G and D
optimizerD = optim.SGD(discriminator.parameters(), lr=0.0002)
optimizerG = optim.SGD(generator.parameters(), lr=0.0002)

In [17]:
for i in range(10):
    
    print('Memory usage when training begins: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    
    print("****************train discriminator*****************")
    encoder_data, discriminator_data, yhat = dloader.get_train_batch(16)

    encoder_data = torch.from_numpy(encoder_data).float().to(device)
    print("encoder_data size: {} bytes => {} GB".format(encoder_data.element_size() * encoder_data.nelement(),
                                                        encoder_data.element_size() * encoder_data.nelement()/1e9))
    
    discriminator_data = torch.from_numpy(discriminator_data).float().to(device)
    print("discriminator_data size: {} bytes => {} GB".format(discriminator_data.element_size() * discriminator_data.nelement(),
                                                              discriminator_data.element_size() * discriminator_data.nelement()/1e9))
    
    yhat = torch.from_numpy(yhat).float().to(device)
    print("yhat size: {} bytes => {} GB".format(yhat.element_size() * yhat.nelement(),
                                                yhat.element_size() * yhat.nelement()/1e9))
    
    # expected_seq = discriminator_data[:, 50:, :, :]
    # print("expected_seq size: {} bytes => {} GB".format(expected_seq.element_size() * expected_seq.nelement(),
    #                                                     expected_seq.element_size() * expected_seq.nelement()/1e9))

    discriminator.zero_grad()
    print('Memory usage after discriminator zero_grad: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    
    d_logits_real = discriminator.forward(discriminator_data, yhat)
    print('Memory usage after d_logits_real forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_real = D_criterion(d_logits_real, torch.ones_like(d_logits_real))
    print('Memory usage after d_loss_real: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_real.backward()
    print('Memory usage after d_loss_real backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    
    predicted_seq, predicted_action, generated_sample = generator.forward(encoder_data, discriminator_data)
    
    print('Memory usage after generator forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    d_logits_fake = discriminator.forward(generated_sample, yhat)
    print('Memory usage after d_logits_fake forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_fake = D_criterion(d_logits_fake, torch.zeros_like(d_logits_fake))
    print('Memory usage after d_loss_fake : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_fake.backward()
    print('Memory usage after d_loss_fake backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

    loss_discriminator = d_loss_real + d_loss_fake
    print('Memory usage after loss_discriminator summed: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    optimizerD.step()
    print('Memory usage after optimizerD stepped: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    
    print("****************train generator*****************")
    encoder_data, discriminator_data, yhat = dloader.get_train_batch(16)

    encoder_data = torch.from_numpy(encoder_data).float().to(device)
    print("encoder_data size: {} bytes => {} GB".format(encoder_data.element_size() * encoder_data.nelement(),
                                                        encoder_data.element_size() * encoder_data.nelement()/1e9))
    
    discriminator_data = torch.from_numpy(discriminator_data).float().to(device)
    print("discriminator_data size: {} bytes => {} GB".format(discriminator_data.element_size() * discriminator_data.nelement(),
                                                              discriminator_data.element_size() * discriminator_data.nelement()/1e9))
    
    yhat = torch.from_numpy(yhat).float().to(device)
    print("yhat size: {} bytes => {} GB".format(yhat.element_size() * yhat.nelement(),
                                                yhat.element_size() * yhat.nelement()/1e9))
    
    expected_seq = discriminator_data[:, 50:, :, :]
    print("expected_seq size: {} bytes => {} GB".format(expected_seq.element_size() * expected_seq.nelement(),
                                                        expected_seq.element_size() * expected_seq.nelement()/1e9))
    
    generator.zero_grad()
    
    predicted_seq, predicted_action, generated_sample = generator.forward(encoder_data, discriminator_data)
    print('Memory usage after generator forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    ReconstructError = G_criterion(predicted_seq, expected_seq)
    print('Memory usage after ReconstructError : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    ReconstructError.backward()
    print('Memory usage after ReconstructError backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    d_logits_fake = discriminator.forward(generated_sample, yhat)
    print('Memory usage after discriminator foward : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    g_loss = D_criterion(d_logits_fake, torch.ones_like(d_logits_fake)) * torch.tensor(0.01)
    print('Memory usage after g_loss : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    # g_loss.backward()
    # print('Memory usage after g_loss backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    loss_generator = ReconstructError + g_loss
    print('Memory usage after loss_generator summed : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    
    optimizerG.step()
    print('Memory usage after optimizerG stepped : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))
    
    # Output training stats
    if i % 1 == 0:
        print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f'
            % (i, 10, loss_discriminator.item(), loss_generator.item()))
    torch.cuda.empty_cache()

Memory usage when training begins: 3133393408 bytes => 3.133393408 GB
encoder_data size: 169344 bytes => 0.000169344 GB
discriminator_data size: 259200 bytes => 0.0002592 GB
yhat size: 960 bytes => 9.6e-07 GB
expected_seq size: 86400 bytes => 8.64e-05 GB
Memory usage after discriminator zero_grad: 3133393408 bytes => 3.133393408 GB
Memory usage after d_logits_real forward: 3522435584 bytes => 3.522435584 GB
Memory usage after d_loss_real: 3522436096 bytes => 3.522436096 GB
Memory usage after d_loss_real backward: 3133393408 bytes => 3.133393408 GB
Memory usage after generator forward: 4569011200 bytes => 4.5690112 GB
Memory usage after d_logits_fake forward: 4957666816 bytes => 4.957666816 GB
Memory usage after d_loss_fake : 4957667328 bytes => 4.957667328 GB
Memory usage after d_loss_fake backward: 3133393408 bytes => 3.133393408 GB
Memory usage after loss_discriminator summed: 3133393408 bytes => 3.133393408 GB
Memory usage after optimizerD stepped: 3133393408 bytes => 3.133393408 GB

KeyboardInterrupt: 

In [10]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 3         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    5721 MB |    7508 MB |   17326 MB |   11604 MB |
|       from large pool |    5687 MB |    7503 MB |   17212 MB |   11524 MB |
|       from small pool |      33 MB |      35 MB |     114 MB |      80 MB |
|---------------------------------------------------------------------------|
| Active memory         |    5721 MB |    7508 MB |   17326 MB |   11604 MB |
|       from large pool |    5687 MB |    7503 MB |   17212 MB |

In [ ]:
encoder_data, discriminator_data, yhat = dloader.get_train_batch(32)
encoder_data = torch.FloatTensor(encoder_data).to('cuda')
discriminator_data = torch.FloatTensor(discriminator_data).to('cuda')
yhat = torch.FloatTensor(yhat).to('cuda')

In [ ]:
lt_encoder = Encoder.Encoder(3,enc_shape=[None, 49, 54, 1], enc_dim_desc={ 'hidden_num': 512,'class_num': 15 })

st_encoder = Encoder.Encoder(3,enc_shape=[None, 20, 54, 1], enc_dim_desc={ 'hidden_num': 512})
decoder = Decoder.Decoder(st_encoder)

generator = ConvSeq2Seq.ConvSeq2Seq(lt_encoder, decoder, window_length=20, device=device)

d_encoder = Encoder.Encoder(32, enc_shape=[None, 75,54,1], enc_dim_desc={ 'hidden_num': 512})
discriminator = Discriminator.Discriminator(32, d_encoder).to(device)

In [ ]:
G_criterion = nn.MSELoss()
D_criterion = nn.BCELoss()

In [ ]:
predicted_seq, predicted_action, generated_sample = generator.forward(encoder_data, discriminator_data)

In [ ]:
expected_seq = discriminator_data[:, 50:, :, :]

In [ ]:
predicted_seq.shape

torch.Size([32, 25, 54, 1])

In [ ]:
expected_seq.shape

torch.Size([32, 25, 54, 1])

In [ ]:
generated_sample.shape

torch.Size([32, 75, 54, 1])

In [ ]:
ReconstructError = G_criterion(predicted_seq, expected_seq)

In [ ]:
d_logits_real = discriminator.forward(discriminator_data, yhat)
d_logits_fake = discriminator.forward(generated_sample, yhat)

In [ ]:
d_loss_real = torch.mean(D_criterion(d_logits_real, torch.ones_like(d_logits_real)))
d_loss_fake = torch.mean(D_criterion(d_logits_fake, torch.zeros_like(d_logits_fake)))
g_loss = torch.mean(D_criterion(d_logits_fake, torch.ones_like(d_logits_fake)))

In [ ]:
loss_d = d_loss_real + d_loss_fake

In [ ]:
loss_d

tensor(1.3915, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
loss_d.backward()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [ ]:
g_loss

tensor(0.6923, device='cuda:0', grad_fn=<MeanBackward0>)

In [30]:
encoder_data, discriminator_data, yhat = dloader.get_train_batch(32)

In [35]:
encoder_data[0][0][0]

array([-0.05696892], dtype=float32)

In [36]:
encoder_data, discriminator_data, yhat = dloader.get_train_batch(32)

In [37]:
encoder_data[0][0][0]

array([-0.14145434], dtype=float32)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()



In [3]:
dloader.train_set[(1,'walking', 1, 'even')].shape

(1738, 54)

In [12]:
a = VAE.EncoderBase(1,2,3,4)

In [21]:
import numpy as np

In [26]:
np.random.randint( 16, 50)

23

In [22]:
np.random.choice(5,3)

array([0, 0, 4])

In [1]:
import torch

In [3]:
torch.float32

torch.float32

In [11]:
for x in range(0, 10, 2):
    print(x)

0
2
4
6
8


In [12]:
import data_utils

In [18]:
filename = '{0}/S{1}/{2}_{3}.txt'.format('./data/h3.6m/dataset', 1, 'walking', 1)
action_sequence = data_utils.readCSVasFloat(filename)

In [19]:
action_sequence.shape

(3476, 99)